<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Inferência-do-INSE" data-toc-modified-id="Inferência-do-INSE-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Inferência do INSE</a></span></li><li><span><a href="#Percentual-de-evasão" data-toc-modified-id="Percentual-de-evasão-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Percentual de evasão</a></span></li><li><span><a href="#Cruzamentos-com-dados-de-aluno" data-toc-modified-id="Cruzamentos-com-dados-de-aluno-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Cruzamentos com dados de aluno</a></span><ul class="toc-item"><li><span><a href="#Sexo" data-toc-modified-id="Sexo-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Sexo</a></span></li><li><span><a href="#Distorção-Idade-Série" data-toc-modified-id="Distorção-Idade-Série-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Distorção Idade-Série</a></span></li><li><span><a href="#Transporte-público" data-toc-modified-id="Transporte-público-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Transporte público</a></span></li><li><span><a href="#Município-da-escola" data-toc-modified-id="Município-da-escola-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Município da escola</a></span></li><li><span><a href="#Tamanho-da-turma" data-toc-modified-id="Tamanho-da-turma-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Tamanho da turma</a></span></li></ul></li><li><span><a href="#Cruzamentos-com-dados-da-escola" data-toc-modified-id="Cruzamentos-com-dados-da-escola-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Cruzamentos com dados da escola</a></span></li></ul></div>

In [1]:
# %load first_cell.py
%reload_ext autoreload
%autoreload 2

from paths import RAW_PATH, TREAT_PATH, OUTPUT_PATH, FIGURES_PATH

import os
from copy import deepcopy
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
import pandas_profiling

import warnings
warnings.filterwarnings('ignore')

# Plotting
import plotly
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

import seaborn as sns
sns.set()

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

colorscale = ['#025951', '#8BD9CA', '#BF7F30', '#F2C124', '#8C470B', '#DFC27D']

In [3]:
df = pd.read_csv(TREAT_PATH / 'analysis' / 'df_alunos.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182838 entries, 0 to 182837
Data columns (total 20 columns):
Unnamed: 0                    182838 non-null int64
ID                            182838 non-null int64
CO_ENTIDADE                   182838 non-null int64
TP_SEXO                       182838 non-null int64
NU_IDADE                      182838 non-null int64
IN_DISTORCAO                  182838 non-null int64
IN_TRANSPORTE_PUBLICO         182838 non-null int64
IN_LOCAL_ESCOLA               182838 non-null int64
N_TURMA                       182838 non-null int64
IN_EVASAO                     182838 non-null int64
NU_ANO_CENSO                  182838 non-null int64
CO_MUNICIPIO                  182838 non-null int64
IN_LABORATORIO_INFORMATICA    182838 non-null float64
IN_LABORATORIO_CIENCIAS       182838 non-null float64
IN_QUADRA_ESPORTES            182838 non-null float64
IN_BIBLIOTECA                 182838 non-null float64
IN_BANHEIRO_FORA_PREDIO       182838 non-null flo

In [4]:
df.head()

,Unnamed: 0,ID,CO_ENTIDADE,TP_SEXO,NU_IDADE,IN_DISTORCAO,IN_TRANSPORTE_PUBLICO,IN_LOCAL_ESCOLA,N_TURMA,IN_EVASAO,NU_ANO_CENSO,CO_MUNICIPIO,IN_LABORATORIO_INFORMATICA,IN_LABORATORIO_CIENCIAS,IN_QUADRA_ESPORTES,IN_BIBLIOTECA,IN_BANHEIRO_FORA_PREDIO,IN_AREA_VERDE,IN_N_COMP_15,NIVEL
0,0,112132289130,33009627,2,17,0,0,2,0,0,2016,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0
1,1,113220290328,33009627,2,21,1,0,2,0,1,2016,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0
2,2,112752673352,33009627,2,16,0,0,2,0,0,2016,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0
3,3,117459545545,33009627,2,17,0,0,2,0,0,2016,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0
4,4,113868062938,33009627,2,18,1,0,2,0,1,2016,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0


In [5]:
dic_categorias = {
    'IN_DISTORCAO': {0: 'Até 17 anos', 1: 'Maior ou igual a 17 anos'},
    'TP_SEXO': {1: 'Homem', 2: 'Mulher'},
    'IN_EVASAO': {0: 'Não evadido', 1: 'Evadido'},
    'IN_TRANSPORTE_PUBLICO': {1: 'Utiliza transporte público', 0: 'Não utiliza'},
    'IN_LOCAL_ESCOLA': {0: 'Rio de Janeiro (cidade)', 1: 'Grande Rio', 2: 'Outros municípios no estado'},
    'N_TURMA': {0: 'Menos de 30 alunos', 1: '30 a 40', 2: 'Mais de 40 alunos'},
    'IN_LABORATORIO_CIENCIAS': {0: 'Escola não possui laboratório de ciências', 1: 'Escola possui laboratório de ciências'},
    'IN_LABORATORIO_INFORMATICA': {0: 'Escola não possui laboratório de informática', 1: 'Escola possui laboratório de informática'},
    'IN_QUADRA_ESPORTES': {0: 'Escola não possui quadra', 1: 'Escola possui quadra'},
    'IN_BIBLIOTECA': {0: 'Escola não possui biblioteca', 1: 'Escola possui biblioteca'},
    'IN_BANHEIRO_FORA_PREDIO': {0: 'Escola não possui banheiro fora do prédio', 1: 'Escola possui banheiro fora do prédio'},
    'IN_AREA_VERDE': {0: 'Escola não possui área verde', 1: 'Escola possui áreaa verde'},
    'IN_N_COMP_15': {0: 'Escola possui menos de 15 computadores', 1: 'Escola possui mais de 15 computadores'},
}

In [6]:
for k,v in dic_categorias.items():
    df['CAT_'+k] = df[k].map(v)

In [7]:
df.head()

,Unnamed: 0,ID,CO_ENTIDADE,TP_SEXO,NU_IDADE,IN_DISTORCAO,IN_TRANSPORTE_PUBLICO,IN_LOCAL_ESCOLA,N_TURMA,IN_EVASAO,NU_ANO_CENSO,CO_MUNICIPIO,IN_LABORATORIO_INFORMATICA,IN_LABORATORIO_CIENCIAS,IN_QUADRA_ESPORTES,IN_BIBLIOTECA,IN_BANHEIRO_FORA_PREDIO,IN_AREA_VERDE,IN_N_COMP_15,NIVEL,CAT_IN_DISTORCAO,CAT_TP_SEXO,CAT_IN_EVASAO,CAT_IN_TRANSPORTE_PUBLICO,CAT_IN_LOCAL_ESCOLA,CAT_N_TURMA,CAT_IN_LABORATORIO_CIENCIAS,CAT_IN_LABORATORIO_INFORMATICA,CAT_IN_QUADRA_ESPORTES,CAT_IN_BIBLIOTECA,CAT_IN_BANHEIRO_FORA_PREDIO,CAT_IN_AREA_VERDE,CAT_IN_N_COMP_15
0,0,112132289130,33009627,2,17,0,0,2,0,0,2016,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0,Até 17 anos,Mulher,Não evadido,Não utiliza,Outros municípios no estado,Menos de 30 alunos,Escola possui laboratório de ciências,Escola possui laboratório de informática,Escola possui quadra,Escola possui biblioteca,Escola possui banheiro fora do prédio,Escola não possui área verde,Escola possui mais de 15 computadores
1,1,113220290328,33009627,2,21,1,0,2,0,1,2016,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0,Maior ou igual a 17 anos,Mulher,Evadido,Não utiliza,Outros municípios no estado,Menos de 30 alunos,Escola possui laboratório de ciências,Escola possui laboratório de informática,Escola possui quadra,Escola possui biblioteca,Escola possui banheiro fora do prédio,Escola não possui área verde,Escola possui mais de 15 computadores
2,2,112752673352,33009627,2,16,0,0,2,0,0,2016,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0,Até 17 anos,Mulher,Não evadido,Não utiliza,Outros municípios no estado,Menos de 30 alunos,Escola possui laboratório de ciências,Escola possui laboratório de informática,Escola possui quadra,Escola possui biblioteca,Escola possui banheiro fora do prédio,Escola não possui área verde,Escola possui mais de 15 computadores
3,3,117459545545,33009627,2,17,0,0,2,0,0,2016,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0,Até 17 anos,Mulher,Não evadido,Não utiliza,Outros municípios no estado,Menos de 30 alunos,Escola possui laboratório de ciências,Escola possui laboratório de informática,Escola possui quadra,Escola possui biblioteca,Escola possui banheiro fora do prédio,Escola não possui área verde,Escola possui mais de 15 computadores
4,4,113868062938,33009627,2,18,1,0,2,0,1,2016,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0,Maior ou igual a 17 anos,Mulher,Evadido,Não utiliza,Outros municípios no estado,Menos de 30 alunos,Escola possui laboratório de ciências,Escola possui laboratório de informática,Escola possui quadra,Escola possui biblioteca,Escola possui banheiro fora do prédio,Escola não possui área verde,Escola possui mais de 15 computadores


##  Inferência do INSE

In [8]:
df['NIVEL'].value_counts(normalize=True) # total dos alunos

3.0    0.865687
4.0    0.123035
2.0    0.007646
5.0    0.002747
6.0    0.000590
1.0    0.000295
Name: NIVEL, dtype: float64

In [9]:
df[['CO_ENTIDADE', 'NIVEL']].drop_duplicates()['NIVEL'].value_counts(normalize=True)

3.0    0.815077
4.0    0.148410
2.0    0.024735
5.0    0.008245
1.0    0.002356
6.0    0.001178
Name: NIVEL, dtype: float64

In [10]:
df[['CO_ENTIDADE', 'NIVEL']].drop_duplicates()['NIVEL'].value_counts()

3.0    692
4.0    126
2.0     21
5.0      7
1.0      2
6.0      1
Name: NIVEL, dtype: int64

In [11]:
df.columns

Index(['Unnamed: 0', 'ID', 'CO_ENTIDADE', 'TP_SEXO', 'NU_IDADE',
       'IN_DISTORCAO', 'IN_TRANSPORTE_PUBLICO', 'IN_LOCAL_ESCOLA', 'N_TURMA',
       'IN_EVASAO', 'NU_ANO_CENSO', 'CO_MUNICIPIO',
       'IN_LABORATORIO_INFORMATICA', 'IN_LABORATORIO_CIENCIAS',
       'IN_QUADRA_ESPORTES', 'IN_BIBLIOTECA', 'IN_BANHEIRO_FORA_PREDIO',
       'IN_AREA_VERDE', 'IN_N_COMP_15', 'NIVEL', 'CAT_IN_DISTORCAO',
       'CAT_TP_SEXO', 'CAT_IN_EVASAO', 'CAT_IN_TRANSPORTE_PUBLICO',
       'CAT_IN_LOCAL_ESCOLA', 'CAT_N_TURMA', 'CAT_IN_LABORATORIO_CIENCIAS',
       'CAT_IN_LABORATORIO_INFORMATICA', 'CAT_IN_QUADRA_ESPORTES',
       'CAT_IN_BIBLIOTECA', 'CAT_IN_BANHEIRO_FORA_PREDIO', 'CAT_IN_AREA_VERDE',
       'CAT_IN_N_COMP_15'],
      dtype='object')

In [12]:
inse = df[['CO_ENTIDADE', 'IN_LOCAL_ESCOLA', 'IN_LABORATORIO_INFORMATICA', 'IN_LABORATORIO_CIENCIAS',
           'IN_QUADRA_ESPORTES', 'IN_BIBLIOTECA', 'IN_BANHEIRO_FORA_PREDIO', 'IN_AREA_VERDE', 
           'IN_N_COMP_15', 'NIVEL']].drop_duplicates()
inse.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1097 entries, 0 to 34258
Data columns (total 10 columns):
CO_ENTIDADE                   1097 non-null int64
IN_LOCAL_ESCOLA               1097 non-null int64
IN_LABORATORIO_INFORMATICA    1097 non-null float64
IN_LABORATORIO_CIENCIAS       1097 non-null float64
IN_QUADRA_ESPORTES            1097 non-null float64
IN_BIBLIOTECA                 1097 non-null float64
IN_BANHEIRO_FORA_PREDIO       1097 non-null float64
IN_AREA_VERDE                 1097 non-null float64
IN_N_COMP_15                  1097 non-null int64
NIVEL                         849 non-null float64
dtypes: float64(7), int64(3)
memory usage: 94.3 KB


In [13]:
print(pd.crosstab(inse['NIVEL'], inse['IN_LOCAL_ESCOLA']))
pd.crosstab(inse['NIVEL'], inse['IN_LOCAL_ESCOLA'], normalize=True)*100

IN_LOCAL_ESCOLA   0    1    2
NIVEL                        
1.0               0    0    2
2.0               1    1   19
3.0              81  363  248
4.0              11   45   70
5.0               0    2    5
6.0               1    0    0


IN_LOCAL_ESCOLA,0,1,2
NIVEL,,,
1.0,0.000000,0.000000,0.235571
2.0,0.117786,0.117786,2.237927
3.0,9.540636,42.756184,29.210836
4.0,1.295642,5.300353,8.244994
5.0,0.000000,0.235571,0.588928
6.0,0.117786,0.000000,0.000000


In [14]:
print(pd.crosstab(inse['NIVEL'], inse['IN_LABORATORIO_INFORMATICA']))
pd.crosstab(inse['NIVEL'], inse['IN_LABORATORIO_INFORMATICA'], normalize=1)*100

IN_LABORATORIO_INFORMATICA  0.0  1.0
NIVEL                               
1.0                           0    2
2.0                           3   18
3.0                         119  573
4.0                          17  109
5.0                           2    5
6.0                           0    1


IN_LABORATORIO_INFORMATICA,0.0,1.0
NIVEL,,
1.0,0.000000,0.282486
2.0,2.127660,2.542373
3.0,84.397163,80.932203
4.0,12.056738,15.395480
5.0,1.418440,0.706215
6.0,0.000000,0.141243


* Todas escolas fora da região metropolitana sem laboratório de informática estão no nível 3. Porém, para outros casos, > 80% também estão nesse nível

In [15]:
compare = [inse['IN_LOCAL_ESCOLA'], inse['IN_LABORATORIO_INFORMATICA']]

print(pd.crosstab(inse['NIVEL'], compare)*100)
pd.crosstab(inse['NIVEL'], compare, normalize=1)*100

IN_LOCAL_ESCOLA                0           1            2       
IN_LABORATORIO_INFORMATICA   0.0   1.0   0.0    1.0   0.0    1.0
NIVEL                                                           
1.0                            0     0     0      0     0    200
2.0                            0   100     0    100   300   1600
3.0                         1600  6500  5800  30500  4500  20300
4.0                            0  1100  1000   3500   700   6300
5.0                            0     0   100    100   100    400
6.0                            0   100     0      0     0      0


IN_LOCAL_ESCOLA                 0                     1                     2  \
IN_LABORATORIO_INFORMATICA    0.0        1.0        0.0        1.0        0.0   
NIVEL                                                                           
1.0                           0.0   0.000000   0.000000   0.000000   0.000000   
2.0                           0.0   1.282051   0.000000   0.292398   5.357143   
3.0                         100.0  83.333333  84.057971  89.181287  80.357143   
4.0                           0.0  14.102564  14.492754  10.233918  12.500000   
5.0                           0.0   0.000000   1.449275   0.292398   1.785714   
6.0                           0.0   1.282051   0.000000   0.000000   0.000000   

IN_LOCAL_ESCOLA                        
IN_LABORATORIO_INFORMATICA        1.0  
NIVEL                                  
1.0                          0.694444  
2.0                          5.555556  
3.0                         70.486111  
4.0                         21.875000  
5.0                          1.388889  
6.0                          0.000000

In [16]:
print(pd.crosstab(inse['NIVEL'], inse['IN_LABORATORIO_CIENCIAS']))
pd.crosstab(inse['NIVEL'], inse['IN_LABORATORIO_CIENCIAS'], normalize=1)*100

IN_LABORATORIO_CIENCIAS  0.0  1.0
NIVEL                            
1.0                        2    0
2.0                       11   10
3.0                      174  518
4.0                       31   95
5.0                        1    6
6.0                        0    1


IN_LABORATORIO_CIENCIAS,0.0,1.0
NIVEL,,
1.0,0.913242,0.000000
2.0,5.022831,1.587302
3.0,79.452055,82.222222
4.0,14.155251,15.079365
5.0,0.456621,0.952381
6.0,0.000000,0.158730


In [17]:
compare = [inse['IN_LABORATORIO_INFORMATICA'], inse['IN_LABORATORIO_CIENCIAS']]
pd.crosstab(inse['NIVEL'], compare, normalize=1)*100

IN_LABORATORIO_INFORMATICA        0.0                   1.0           
IN_LABORATORIO_CIENCIAS           0.0        1.0        0.0        1.0
NIVEL                                                                 
1.0                          0.000000   0.000000   1.075269   0.000000
2.0                          9.090909   0.000000   4.301075   1.915709
3.0                         75.757576  87.037037  80.107527  81.226054
4.0                         15.151515  11.111111  13.978495  15.900383
5.0                          0.000000   1.851852   0.537634   0.766284
6.0                          0.000000   0.000000   0.000000   0.191571

In [18]:
pd.crosstab(inse['NIVEL'], inse['IN_BIBLIOTECA'], normalize=1)*100

IN_BIBLIOTECA,0.0,1.0
NIVEL,,
1.0,1.428571,0.128370
2.0,2.857143,2.439024
3.0,84.285714,81.258023
4.0,10.000000,15.275995
5.0,1.428571,0.770218
6.0,0.000000,0.128370


In [19]:
compare = [inse['IN_BIBLIOTECA'], inse['IN_LABORATORIO_CIENCIAS']]
pd.crosstab(inse['NIVEL'], compare, normalize=1)*100

IN_BIBLIOTECA             0.0              1.0           
IN_LABORATORIO_CIENCIAS   0.0   1.0        0.0        1.0
NIVEL                                                    
1.0                       2.0   0.0   0.591716   0.000000
2.0                       4.0   0.0   5.325444   1.639344
3.0                      86.0  80.0  77.514793  82.295082
4.0                       8.0  15.0  15.976331  15.081967
5.0                       0.0   5.0   0.591716   0.819672
6.0                       0.0   0.0   0.000000   0.163934

In [20]:
pd.crosstab(inse['NIVEL'], inse['IN_BANHEIRO_FORA_PREDIO'], normalize=1)*100

IN_BANHEIRO_FORA_PREDIO,0.0,1.0
NIVEL,,
1.0,1.020408,0.000000
2.0,2.551020,2.450230
3.0,77.040816,82.848392
4.0,17.857143,13.935681
5.0,1.020408,0.765697
6.0,0.510204,0.000000


In [21]:
compare = [inse['IN_BIBLIOTECA'], inse['IN_BANHEIRO_FORA_PREDIO']]
pd.crosstab(inse['NIVEL'], compare, normalize=1)*100

IN_BIBLIOTECA                  0.0              1.0           
IN_BANHEIRO_FORA_PREDIO        0.0   1.0        0.0        1.0
NIVEL                                                         
1.0                       2.222222   0.0   0.662252   0.000000
2.0                       2.222222   4.0   2.649007   2.388535
3.0                      88.888889  76.0  73.509934  83.121019
4.0                       4.444444  20.0  21.854305  13.694268
5.0                       2.222222   0.0   0.662252   0.796178
6.0                       0.000000   0.0   0.662252   0.000000

In [22]:
pd.crosstab(inse['NIVEL'], inse['IN_N_COMP_15'], normalize=1)*100

IN_N_COMP_15,0,1
NIVEL,,
1.0,0.313480,0.000000
2.0,2.664577,1.895735
3.0,81.347962,81.990521
4.0,14.576803,15.639810
5.0,1.097179,0.000000
6.0,0.000000,0.473934


In [23]:
compare = [inse['IN_LABORATORIO_INFORMATICA'], inse['IN_LABORATORIO_CIENCIAS'], inse['IN_N_COMP_15']]
           # inse['IN_BIBLIOTECA'], inse['IN_BANHEIRO_FORA_PREDIO'], inse['IN_QUADRA_ESPORTES']]

pd.crosstab(inse['NIVEL'], compare, normalize=1)*100

IN_LABORATORIO_INFORMATICA     0.0                                 1.0  \
IN_LABORATORIO_CIENCIAS        0.0               1.0               0.0   
IN_N_COMP_15                     0      1          0      1          0   
NIVEL                                                                    
1.0                          0.000    0.0   0.000000    0.0   1.526718   
2.0                          9.375    0.0   0.000000    0.0   5.343511   
3.0                         78.125    0.0  86.792453  100.0  79.389313   
4.0                         12.500  100.0  11.320755    0.0  12.977099   
5.0                          0.000    0.0   1.886792    0.0   0.763359   
6.0                          0.000    0.0   0.000000    0.0   0.000000   

IN_LABORATORIO_INFORMATICA                                   
IN_LABORATORIO_CIENCIAS                      1.0             
IN_N_COMP_15                        1          0          1  
NIVEL                                                        
1.0                          0.000000   0.000000   0.000000  
2.0                          1.818182   1.897019   1.960784  
3.0                         81.818182  80.758808  82.352941  
4.0                         16.363636  16.260163  15.032680  
5.0                          0.000000   1.084011   0.000000  
6.0                          0.000000   0.000000   0.653595

In [24]:
compare = [inse['IN_BIBLIOTECA'], inse['IN_BANHEIRO_FORA_PREDIO'], inse['IN_QUADRA_ESPORTES']]

pd.crosstab(inse['NIVEL'], compare, normalize=1)*100

IN_BIBLIOTECA                  0.0                                    1.0  \
IN_BANHEIRO_FORA_PREDIO        0.0              1.0                   0.0   
IN_QUADRA_ESPORTES             0.0        1.0   0.0        1.0        0.0   
NIVEL                                                                       
1.0                       5.555556   0.000000   0.0   0.000000   0.000000   
2.0                       5.555556   0.000000  25.0   0.000000   2.631579   
3.0                      77.777778  96.296296  50.0  80.952381  73.684211   
4.0                       5.555556   3.703704  25.0  19.047619  21.052632   
5.0                       5.555556   0.000000   0.0   0.000000   2.631579   
6.0                       0.000000   0.000000   0.0   0.000000   0.000000   

IN_BIBLIOTECA                                             
IN_BANHEIRO_FORA_PREDIO                   1.0             
IN_QUADRA_ESPORTES             1.0        0.0        1.0  
NIVEL                                                     
1.0                       0.884956   0.000000   0.000000  
2.0                       2.654867   7.142857   2.280130  
3.0                      73.451327  71.428571  83.387622  
4.0                      22.123894  21.428571  13.517915  
5.0                       0.000000   0.000000   0.814332  
6.0                       0.884956   0.000000   0.000000

In [25]:
pd.crosstab(inse['NIVEL'], compare)

IN_BIBLIOTECA           0.0             1.0             
IN_BANHEIRO_FORA_PREDIO 0.0     1.0     0.0     1.0     
IN_QUADRA_ESPORTES      0.0 1.0 0.0 1.0 0.0 1.0 0.0  1.0
NIVEL                                                   
1.0                       1   0   0   0   0   1   0    0
2.0                       1   0   1   0   1   3   1   14
3.0                      14  26   2  17  28  83  10  512
4.0                       1   1   1   4   8  25   3   83
5.0                       1   0   0   0   1   0   0    5
6.0                       0   0   0   0   0   1   0    0

## Percentual de evasão

In [26]:
v = df['CAT_IN_EVASAO'].value_counts(normalize=True)

fig = v.iplot(kind='bar', color=colorscale[1], 
              title='Percentual de alunos do 1º ano evadidos e não evadidos (2016 para 2017)')

## Cruzamentos com dados de aluno

In [27]:
def plot_dist(x, title2, save=True):
    
    v_cat = df[x].value_counts(normalize=True)
    data = [go.Bar({'x': v_cat.index, 'y': v_cat, 'marker': {'color': colorscale}})]

    fig = {'data': data, 'layout': {'title': title2}}
    
    if save:
        plotly.offline.iplot(fig, image='png', filename='dist_'+x)
    else: 
        plotly.offline.iplot(fig)
    # plotly.offline.iplot({'data': data, 'layout': {'title': title2}}, image='png', filename='dist_'+x)

def plot_categoria(x, title1, filename, ambos=True, title2=None, save=True):
    
    if ambos:
        # Distribuição total
        plot_dist(x, title2, save)
        
    # Distribuição com evasão
    data = []
    i = 0
    for label in df[x].unique():
        
        v_cat = df[df[x] == label]['CAT_IN_EVASAO'].value_counts(normalize=True)

        data.append(go.Bar({'x': v_cat.index, 'y': v_cat, 'name': label,
                           'marker': {'color': colorscale[i]}}))
        i += 1
        
    # Adicionando total
    total = df['CAT_IN_EVASAO'].value_counts(normalize=True)
    data.append(go.Bar({'x': total.index, 'y': total, 'name': 'Total', 
                        'marker': {'color': '#dbdbdb'}}))
    
    fig = {'data': data, 'layout': {'title': title1}}
    
    if save:
        plotly.offline.iplot(fig, image='png', filename=filename)
    else: 
        plotly.offline.iplot(fig)

In [28]:
def cross_evasao_sexo(x, y='CAT_TP_SEXO'):
    t = pd.crosstab([df[x], df[y]], df['CAT_IN_EVASAO'], normalize=0)*100
    t.columns.name = None
    return t

def plot_categoria_sexo(x, title, filename, y='CAT_TP_SEXO', save=True):
    
    v = cross_evasao_sexo(x, y).reset_index()
        
    # Distribuição com evasão
    data = []
    i = 0
    for label in v[x].unique():

        v_cat = v[v[x] == label]
        data.append(go.Bar({'x': v_cat[y], 'y': v_cat['Evadido'], 'name': label,
                           'marker': {'color': colorscale[i]}}))
        i += 1
    
    fig = {'data': data, 'layout': {'title': title}}
    
    if save:
        plotly.offline.iplot(fig, image='png', filename=filename)
    else: 
        plotly.offline.iplot(fig)
        
    # plotly.offline.iplot({'data': data, 'layout': {'title': title}}, image='png', filename=filename)
    return v

### Sexo

In [29]:
# def plot_perc_categoria(x):
    
#     v = df[['ID', 'IN_EVASAO', x]].groupby(['IN_EVASAO', x]).sum()
#     return v

In [30]:
df['CAT_TP_SEXO'].unique()

array(['Mulher', 'Homem'], dtype=object)

In [31]:
title = 'Dentre as mulheres e homens, qual percentual evade?'
x = 'CAT_TP_SEXO'
categorias = {1: 'Homens', 2: 'Mulheres'}

plot_categoria(x, title, 'perc_sexo_evasao', title2='Distribuição de sexo entre alunos', save=False)

In [32]:
# v_sexo = df[df['TP_SEXO'] == 1]['IN_EVASAO'].value_counts(normalize=True)

# v_sexo.index = v_sexo.index.map({1: 'Evadidos', 0: 'Não evadidos'})

# data = [go.Bar({'x': v_sexo.index, 
#                 'y': v_sexo, 
#                 'name': 'Homens'}), 
#         go.Bar({'x': v.index, 
#                 'y': v, 
#                 'name': 'Total'})]

# plotly.offline.iplot({'data': data, 'layout': {'title': 'Dentre os homens, qual percentual evade?'}})

In [33]:
# v_sexo = df[['ID', 'IN_EVASAO', 'TP_SEXO']].groupby(['IN_EVASAO', 'TP_SEXO']).count() # dentre evadidos, quantas mulheres?
# n = v_sexo.groupby('IN_EVASAO').sum()

# v_sexo/n

# data = [go.Bar({'x': v_sexo.index, 
#                 'y': v_sexo, 
#                 'name': 'Mulheres'}), 
#         go.Bar({'x': v.index, 
#                 'y': v, 
#                 'name': 'Total'})]

# plotly.offline.iplot({'data': data, 'layout': {'title': 'Dentre evadidos, quantas são mulheres?'}})

### Distorção Idade-Série

In [34]:
title = 'Dentre alunos com e sem distorção idade-série, qual percentual evade?'
x = 'CAT_IN_DISTORCAO'

plot_categoria_sexo(x, title, 'perc_sexo_idade_evasao', save=False)

,CAT_IN_DISTORCAO,CAT_TP_SEXO,Evadido,Não evadido
0,Até 17 anos,Homem,23.122745,76.877255
1,Até 17 anos,Mulher,22.526101,77.473899
2,Maior ou igual a 17 anos,Homem,59.219295,40.780705
3,Maior ou igual a 17 anos,Mulher,56.366778,43.633222


In [35]:
title = 'Dentre alunos com e sem distorção idade-série, qual percentual evade?'
x = 'CAT_IN_DISTORCAO'
categorias = {1: 'Maior ou igual a 17 anos', 0: 'Até 17 anos'}

plot_categoria(x, title, 'perc_distorcao_evasao', 
               title2='Percentual de alunos com e sem distorção idade-série', save=False)

### Transporte público

In [36]:
v = cross_evasao_sexo('CAT_IN_TRANSPORTE_PUBLICO').reset_index()
v

,CAT_IN_TRANSPORTE_PUBLICO,CAT_TP_SEXO,Evadido,Não evadido
0,Não utiliza,Homem,42.841745,57.158255
1,Não utiliza,Mulher,38.883217,61.116783
2,Utiliza transporte público,Homem,33.009080,66.990920
3,Utiliza transporte público,Mulher,28.800744,71.199256


In [37]:
title = 'Dentre alunos que utilizam ou não transporte público, qual percentual evade?'
x = 'CAT_IN_TRANSPORTE_PUBLICO'

plot_categoria_sexo(x, title, 'perc_sexo_transporte_evasao', save=False)

,CAT_IN_TRANSPORTE_PUBLICO,CAT_TP_SEXO,Evadido,Não evadido
0,Não utiliza,Homem,42.841745,57.158255
1,Não utiliza,Mulher,38.883217,61.116783
2,Utiliza transporte público,Homem,33.009080,66.990920
3,Utiliza transporte público,Mulher,28.800744,71.199256


In [38]:
title = 'Dentre alunos que utilizam ou não transporte público, qual percentual evade?'
x = 'CAT_IN_TRANSPORTE_PUBLICO'

plot_categoria(x, title, 'perc_transporte_evasao', title2='Percentual de alunos por tipo de transporte', save=False)

### Município da escola

In [39]:
cross_evasao_sexo('CAT_IN_LOCAL_ESCOLA')

Evadido  Não evadido
CAT_IN_LOCAL_ESCOLA         CAT_TP_SEXO                        
Grande Rio                  Homem        33.288517    66.711483
                            Mulher       28.494898    71.505102
Outros municípios no estado Homem        34.530864    65.469136
                            Mulher       29.496369    70.503631
Rio de Janeiro (cidade)     Homem        40.609709    59.390291
                            Mulher       37.530117    62.469883

In [40]:
x = 'CAT_IN_LOCAL_ESCOLA'
y = 'CAT_IN_TRANSPORTE_PUBLICO'
plot_categoria_sexo(x, title, 'perc_cross_transp_local_evasao', y, save=False)

,CAT_IN_LOCAL_ESCOLA,CAT_IN_TRANSPORTE_PUBLICO,Evadido,Não evadido
0,Grande Rio,Não utiliza,41.293630,58.706370
1,Grande Rio,Utiliza transporte público,26.844346,73.155654
2,Outros municípios no estado,Não utiliza,34.976538,65.023462
3,Outros municípios no estado,Utiliza transporte público,29.464999,70.535001
4,Rio de Janeiro (cidade),Não utiliza,46.444904,53.555096
5,Rio de Janeiro (cidade),Utiliza transporte público,36.005336,63.994664


In [41]:
title = 'Dentre os locais que os alunos estudam, qual percentual evade?'
x = 'CAT_IN_LOCAL_ESCOLA'

plot_categoria(x, title, 'perc_cidade_evasao', title2='Percentual de alunos por município', save=False)

### Tamanho da turma

In [42]:
df['N_TURMA'].value_counts()

1    87925
0    60450
2    34463
Name: N_TURMA, dtype: int64

In [43]:
title = 'Dentre os locais que os alunos estudam, qual percentual evade?'
x = 'CAT_N_TURMA'
y = 'CAT_IN_LOCAL_ESCOLA'

plot_categoria_sexo(x, title, 'perc_cross_turma_local_evasao', y, save=False)

,CAT_N_TURMA,CAT_IN_LOCAL_ESCOLA,Evadido,Não evadido
0,30 a 40,Grande Rio,29.937127,70.062873
1,30 a 40,Outros municípios no estado,31.805749,68.194251
2,30 a 40,Rio de Janeiro (cidade),39.412052,60.587948
3,Mais de 40 alunos,Grande Rio,32.085756,67.914244
4,Mais de 40 alunos,Outros municípios no estado,34.625045,65.374955
5,Mais de 40 alunos,Rio de Janeiro (cidade),38.142613,61.857387
6,Menos de 30 alunos,Grande Rio,31.975925,68.024075
7,Menos de 30 alunos,Outros municípios no estado,31.908214,68.091786
8,Menos de 30 alunos,Rio de Janeiro (cidade),40.398724,59.601276


In [44]:
title = 'Dentre os locais que os alunos estudam, qual percentual evade?'
x = 'CAT_N_TURMA'

plot_categoria_sexo(x, title, 'perc_cross_turma_sexo_evasao', save=False)

,CAT_N_TURMA,CAT_TP_SEXO,Evadido,Não evadido
0,30 a 40,Homem,35.880318,64.119682
1,30 a 40,Mulher,31.716427,68.283573
2,Mais de 40 alunos,Homem,38.333053,61.666947
3,Mais de 40 alunos,Mulher,34.340031,65.659969
4,Menos de 30 alunos,Homem,36.036899,63.963101
5,Menos de 30 alunos,Mulher,31.183682,68.816318


In [45]:
title = 'Qual o percentual de evasão por tamanho de turma?'

x = 'CAT_N_TURMA'
plot_categoria(x, title, 'perc_n_turma_evasao', title2='Percentual de alunos por tamanho da turma', save=False)

## Cruzamentos com dados da escola

In [46]:
title = 'Dentre alunos de escolas com laboratório de informática, qual percentual evade?'
x = 'CAT_IN_LABORATORIO_INFORMATICA'

plot_categoria(x, title, 'perc_lab_inform_evasao', 
               title2='Distribuição alunos em escolas com e sem laboratório de informática', save=False)

In [47]:
title = 'Dentre alunos de escolas com laboratório de ciências, qual percentual evade?'
x = 'CAT_IN_LABORATORIO_CIENCIAS'

plot_categoria(x, title, 'perc_lab_ciencias_evasao',
               title2='Distribuição alunos em escolas com e sem laboratório de ciências', save=False)

In [48]:
title = 'Dentre alunos de escolas com quadra, qual percentual evade?'
x = 'CAT_IN_QUADRA_ESPORTES'

plot_categoria(x, title, 'perc_quadra_evasao', 
               title2='Distribuição alunos em escolas com e sem quadra', save=False)

In [49]:
title = 'Dentre alunos de escolas com biblioteca, qual percentual evade?'
x = 'CAT_IN_BIBLIOTECA'

plot_categoria(x, title, 'perc_biblioteca_evasao', 
               title2='Distribuição alunos em escolas com e sem bibioteca', save=False)

In [50]:
title = 'Dentre alunos de escolas com banheiro fora do prédio, qual percentual evade?'
x = 'CAT_IN_BANHEIRO_FORA_PREDIO'

plot_categoria(x, title, 'perc_ban_fora_evasao', 
               title2='Distribuição alunos em escolas com e sem banheiro fora do prédio', save=False)

In [51]:
title = 'Dentre alunos de escolas com área verde, qual percentual evade?'
x = 'CAT_IN_AREA_VERDE'

plot_categoria(x, title, 'perc_verde_evasao', 
               title2='Distribuição alunos em escolas com e sem banheiro fora do prédio', save=False)

In [52]:
title = 'Dentre alunos de escolas com computadores disponíveis, qual percentual evade?'
x = 'CAT_IN_N_COMP_15'

plot_categoria(x, title, 'perc_verde_evasao', 
               title2='Distribuição alunos em escolas com e sem computadores', save=False)